## 3.4 엑셀 통합 문서에서 워크시트 집합 읽기

### 3.4.1 통합 문서에서 워크시트의 일부분만 처리할 때

In [359]:
import sys
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_file = "./sales_2013.xlsx"
output_file = "11output.xls"

output_workbook = Workbook() #데이터를 쌓을 빈 워크북을 만든다
output_worksheet = output_workbook.add_sheet('set_of_worksheets') 
#위의 워크북에 'set_of_worksheets'라는 이름의 워크시트를 추가한다

my_sheets = [0,1]
threshold = 1900.0
sales_column_index = 3


first_worksheet = True #첫 번째 워크시트가 참인 상태
with open_workbook(input_file) as workbook:
    data = [] #아래 for문을 통해 필터링된 데이터를 추가할 빈 리스트
    for sheet_index in range(workbook.nsheets): #가져온 엑셀 파일에 있는 워크시트 중에서
        if sheet_index in my_sheets: #(my_sheets = [0,1])범위 안에서
            worksheet = workbook.sheet_by_index(sheet_index) #워크시트 위치를 기준으로 가져온다.
            if first_worksheet: #그리고 첫 번째 워크시트 = True일 때
                header_row = worksheet.row_values(0) #워크시트의 0번 row_value(첫 번째 행)을 header_row로 할당한다.
                data.append(header_row) #header_row를 위의 데이터 리스트에 추가한다
                first_worksheet = False #헤더를 가져오는 작업이 끝났으므로 상태를 변경한다.
                
                #print("header_row:",header_row)
                #print("data:",data)
                
            for row_index in range(1,worksheet.nrows): #헤더 다음 행부터
                row_list = []
                sale_amount = worksheet.cell_value(row_index, sales_column_index)
                #빈 row_list와 sale_amount를 가져올 셀 위치(행, 열) 지정
                if sale_amount > threshold:
                    for column_index in range(worksheet.ncols):
                        cell_value = worksheet.cell_value(row_index,column_index) 
                        #sale_amount가 1900 초과할 때, cell_value에 행,열 위치를 기록한다
                        cell_type = worksheet.cell_type(row_index,column_index) 
                        #각 위치의 데이터 타입을 기록한다
                        if cell_type == 3: #type 3은 날짜
                            date_cell = xldate_as_tuple(cell_value,workbook.datemode)
                            date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                            #데이터 타입이 날짜일 때, 월/일/연 형태의 날짜 데이터로 변환해 date_cell에 기록한다
                            row_list.append(date_cell) #date_cell을 row_list에 추가한다.
                        else:
                            row_list.append(cell_value) #날짜가 아닌 셀 값을 row_list에 추가한다.
                    if row_list:
                        data.append(row_list)
                        #유효한 row_list를 data 리스트에 추가한다
    print(data)
    
    print("\n행, 열, 내용 : ")
    for list_index, output_list in enumerate(data):
        for element_index, element in enumerate(output_list):
            output_worksheet.write(list_index, element_index, element)
            print(list_index, element_index, element)
    
output_workbook.save(output_file)

pd.read_excel("11output.xls")

[['Customer ID', 'Customer Name', 'Invoice Number', 'Sale Amount', 'Purchase Date'], [6789.0, 'Samantha Donaldson', '100-0007', 1995.0, '01/31/2013'], [7654.0, 'Roger Lipney', '100-0010', 2135.0, '02/15/2013']]

행, 열, 내용 : 
0 0 Customer ID
0 1 Customer Name
0 2 Invoice Number
0 3 Sale Amount
0 4 Purchase Date
1 0 6789.0
1 1 Samantha Donaldson
1 2 100-0007
1 3 1995.0
1 4 01/31/2013
2 0 7654.0
2 1 Roger Lipney
2 2 100-0010
2 3 2135.0
2 4 02/15/2013


,Customer ID,Customer Name,Invoice Number,Sale Amount,Purchase Date
0,6789,Samantha Donaldson,100-0007,1995,01/31/2013
1,7654,Roger Lipney,100-0010,2135,02/15/2013


#### 팬더스

- read_excel() 함수에서 인덱스 번호나 이름으로 워크시트 선택

In [360]:
import pandas as pd

input_file = "./sales_2013.xlsx"
output_file = "11pandas_output.xls"

my_sheets = [0,1]
threshold = 1900.0

data_frame = pd.read_excel(input_file, sheetname=my_sheets, index_col=None)

row_list = []
for worksheet_name, data in data_frame.items():
    row_list.append(data[data['Sale Amount'].astype(float) > threshold])
    # data_frame 안의 워크시트 순서대로 데이터를 가져오고 float타입으로 변환한 'Sale Amount' 값에 따라 row_list에 추가함
print(worksheet_name, row_list)

filtered_rows = pd.concat(row_list, axis=0, ignore_index=True)
#필터링된 행을 하나의 데이터로 합침

writer = pd.ExcelWriter(output_file)
filtered_rows.to_excel(writer, sheet_name='set_of_worksheets',index=False)
#to_excel 함수에서 저장할 시트 이름을 지정한다.
writer.save()

pd.read_excel("11pandas_output.xls")

1 [   Customer ID       Customer Name Invoice Number  Sale Amount Purchase Date
5         6789  Samantha Donaldson       100-0007         1995    2013-01-31,    Customer ID Customer Name Invoice Number  Sale Amount Purchase Date
2         7654  Roger Lipney       100-0010         2135    2013-02-15]


,Customer ID,Customer Name,Invoice Number,Sale Amount,Purchase Date
0,6789,Samantha Donaldson,100-0007,1995,2013-01-31
1,7654,Roger Lipney,100-0010,2135,2013-02-15


## 3.5 여러 개의 통합문서 처리하기
### 3.5.1 통합 문서의 개수 및 각 통합 문서의 행과 열 개수 세기

In [355]:
import glob
import os
from xlrd import open_workbook

input_directory = "./"

workbook_counter = 0
for input_file in glob.glob(os.path.join(input_directory, '*.xlsx')):
    workbook = open_workbook(input_file)
    print('Workbook: {}'.format(os.path.basename(input_file)))
    print('number of worksheets: {}'.format(workbook.nsheets))
    for worksheet in workbook.sheets():
        print('Worksheet name:', worksheet.name, '\tRows',\
             worksheet.nrows, '\tColumns:', worksheet.ncols)
    print('\r')
    workbook_counter += 1
print('Number of Excel workbooks: {}'.format(workbook_counter))

Workbook: sales_2013.xlsx
number of worksheets: 3
Worksheet name: january_2013 	Rows 7 	Columns: 5
Worksheet name: february_2013 	Rows 7 	Columns: 5
Worksheet name: march_2013 	Rows 7 	Columns: 5

Workbook: sales_2014.xlsx
number of worksheets: 3
Worksheet name: january_2014 	Rows 7 	Columns: 5
Worksheet name: february_2014 	Rows 7 	Columns: 5
Worksheet name: march_2014 	Rows 7 	Columns: 5

Workbook: sales_2015.xlsx
number of worksheets: 3
Worksheet name: january_2015 	Rows 7 	Columns: 5
Worksheet name: february_2015 	Rows 7 	Columns: 5
Worksheet name: march_2015 	Rows 7 	Columns: 5

Number of Excel workbooks: 3


### 3.5.2 여러 개의 통합 문서 합치기

#### 기본 파이썬

In [351]:
import glob
import os
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_folder = "./"
output_file = "12output.xls"

output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('all_data_all_workbooks')

data = []
first_worksheet = True

for input_file in glob.glob(os.path.join(input_folder, '*.xlsx')):
    print(os.path.basename(input_file))
    with open_workbook(input_file) as workbook:
        for worksheet in workbook.sheets():
            if first_worksheet:
                header_row = worksheet.row_values(0) #0번째 행의 데이터를 헤더로 저장
                data.append(header_row)
                first_worksheet = False
                #print(data)
                
            for row_index in range(1,worksheet.nrows):
                row_list = []
                for column_index in range(worksheet.ncols):
                    cell_value = worksheet.cell_value(row_index, column_index) #행, 열 위치에 따라 cell 값 지정
                    cell_type = worksheet.cell_type(row_index, column_index)
                    if cell_type == 3:
                        date_cell = xldate_as_tuple(cell_value, workbook.datemode)
                        date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                        row_list.append(date_cell)
                    else:
                        row_list.append(cell_value)
                data.append(row_list)

for list_index, output_list in enumerate(data):
    for element_index, element in enumerate(output_list):
        output_worksheet.write(list_index, element_index, element)

output_workbook.save(output_file)

pd.read_excel("12output.xls").head(11)

sales_2013.xlsx
sales_2014.xlsx
sales_2015.xlsx


,Customer ID,Customer Name,Invoice Number,Sale Amount,Purchase Date
43,8765,Laney Stone,100-0009,34665,02/08/2015
44,7654,Roger Lipney,100-0010,86,02/15/2015
45,6543,Thomas Haines,100-0011,2353,02/17/2015
46,5432,Anushka Vaz,100-0012,5687,02/21/2015
47,4321,Harriet Cooper,100-0013,973,02/25/2015
48,1234,John Smith,100-0014,123,03/04/2015
49,8765,Tony Song,100-0015,234435,03/08/2015
50,2345,Mary Harrison,100-0016,456,03/17/2015
51,6543,Rachel Paz,100-0017,6476,03/22/2015
52,3456,Lucy Gomez,100-0018,234,03/28/2015


#### 팬더스

In [364]:
import pandas as pd
import glob
import os

input_path = "./"
output_file = "12pandas_output.xls"

all_workbooks = glob.glob(os.path.join(input_path, '*.xlsx'))
data_frames = [] #예제의 변수명을 보니 뭔가를 데이터프레임 형태로 저장할 것 같은 기운이 온다

for workbook in all_workbooks:
    all_worksheets = pd.read_excel(workbook, sheetname=None, index_col=None)
    for worksheet_name, data in all_worksheets.items(): #워크시트 이름과 데이터를 묶어 아이템으로 가져온다
        data_frames.append(data)
        #각 워크시트 별 데이터를 묶어 data_frames로 저장한다
    
all_data_concatenated = pd.concat(data_frames, axis=0, ignore_index=None)
#각각의 데이터 프레임을 세로로 합친다
#print(all_data_concatenated)

writer = pd.ExcelWriter(output_file)
all_data_concatenated.to_excel(writer, sheet_name='all_data_all_workbooks', index=False)
writer.save()

pd.read_excel("12pandas_output.xls").tail(11)

,Customer ID,Customer Name,Invoice Number,Sale Amount,Purchase Date
43,8765,Laney Stone,100-0009,34665,2015-02-08
44,7654,Roger Lipney,100-0010,86,2015-02-15
45,6543,Thomas Haines,100-0011,2353,2015-02-17
46,5432,Anushka Vaz,100-0012,5687,2015-02-21
47,4321,Harriet Cooper,100-0013,973,2015-02-25
48,1234,John Smith,100-0014,123,2015-03-04
49,8765,Tony Song,100-0015,234435,2015-03-08
50,2345,Mary Harrison,100-0016,456,2015-03-17
51,6543,Rachel Paz,100-0017,6476,2015-03-22
52,3456,Lucy Gomez,100-0018,234,2015-03-28


### 3.5.3 통합 문서 및 워크시트별 합계 및 평균 계산하기

#### 기본 파이썬

In [350]:
import glob
import os
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_folder = "./"
output_file = "13output.xls"

output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('sums_and_averages')

all_data = []
sales_column_index = 3

header = ['workbook', 'worksheet', 'worksheet_total', 'worksheet_average', 'workbook_total', 'workbook_average']
all_data.append(header)

for input_file in glob.glob(os.path.join(input_path, '*.xlsx')):
    with open_workbook(input_file) as workbook:
        list_of_totals = []
        list_of_numbers = []
        workbook_output = []
        
        for worksheet in workbook.sheets():
            total_sales = 0
            number_of_sales = 0
            worksheet_list = []
            worksheet_list.append(os.path.basename(input_file))
            worksheet_list.append(worksheet.name)
            #print(worksheet_list) # worksheet_list에는 파일 이름과 시트 이름이 들어간다
            
            for row_index in range(1, worksheet.nrows):
                try:
                    total_sales += float(str(worksheet.cell_value (row_index, sales_column_index)).strip('$').replace(',',''))
                    number_of_sales += 1. #이 마침표는 뭘까
                except:
                    total_sales += 0.
                    number_of_sales += 0.
                # try-except 구문
                # 결과가 유효한 값만 계산한다

            average_sales = '%.2f' % (total_sales / number_of_sales)
            worksheet_list.append(total_sales)
            worksheet_list.append(float(average_sales))
            list_of_totals.append(total_sales)
            list_of_numbers.append(float(number_of_sales))
            workbook_output.append(worksheet_list)
        workbook_total = sum(list_of_totals)
        workbook_average = sum(list_of_totals)/sum(list_of_numbers)

        for list_element in workbook_output:
            list_element.append(workbook_total)
            list_element.append(workbook_average)
        all_data.extend(workbook_output)
        #append와 extend의 차이점
        #append는 리스트를 그대로 추가하고, extend는 리스트의 각 element를 추가한다
        #print(all_data)
    
for list_index, output_list in enumerate(all_data):
    for element_index, element in enumerate(output_list):
        output_worksheet.write(list_index, element_index, element)
        
output_workbook.save(output_file)

pd.read_excel("13output.xls")

,workbook,worksheet,worksheet_total,worksheet_average,workbook_total,workbook_average
0,sales_2013.xlsx,january_2013,8992,1498.67,28506,1583.666667
1,sales_2013.xlsx,february_2013,9375,1562.50,28506,1583.666667
2,sales_2013.xlsx,march_2013,10139,1689.83,28506,1583.666667
3,sales_2014.xlsx,january_2014,260221,43370.17,465386,25854.777778
4,sales_2014.xlsx,february_2014,103656,17276.00,465386,25854.777778
5,sales_2014.xlsx,march_2014,101509,16918.17,465386,25854.777778
6,sales_2015.xlsx,january_2015,3201,533.50,304253,16902.944444
7,sales_2015.xlsx,february_2015,55007,9167.83,304253,16902.944444
8,sales_2015.xlsx,march_2015,246045,41007.50,304253,16902.944444


In [280]:
list1 = [1,2,3,4,5]
list2 = [6,7]

list1.append(list2)
print(list1)

[1, 2, 3, 4, 5, [6, 7]]


In [281]:
list1 = [1,2,3,4,5]
list2 = [6,7]

list1.extend(list2)
print(list1)

[1, 2, 3, 4, 5, 6, 7]


#### 팬더스

In [324]:
import pandas as pd
import glob
import os

input_path = "./"
output_file = "13pandas_output.xls"

all_workbooks = glob.glob(os.path.join(input_path, '*.xlsx'))
data_frames = []

for workbook in all_workbooks:
    all_worksheets = pd.read_excel(workbook, sheetname=None, index_col=None)
    workbook_total_sales = []
    workbook_number_of_sales = []
    worksheet_data_frames = []
    worksheets_data_frame = None
    workbook_data_frame = None
    #빈 리스트는 []로 시작하고, 빈 데이터프레임은 None으로 시작한다
    
    for worksheet_name, data in all_worksheets.items():
        total_sales = pd.DataFrame([float(str(value).strip('$').replace(',','')) \
                                          for value in data.loc[:, 'Sale Amount']]).sum()
        number_of_sales = len(data.loc[:,'Sale Amount'])
        average_sales = pd.DataFrame(total_sales / number_of_sales)
        
        workbook_total_sales.append(total_sales)
        workbook_number_of_sales.append(number_of_sales)
        
        data = {'workbook': os.path.basename(workbook),
                'worksheet': worksheet_name,
                'worksheet_total': total_sales,
                'worksheet_average': average_sales}
        #for 문이 돌면서 계산한 데이터들을 키-값으로 묶어 data 딕셔너리에 저장한다
        
        worksheet_data_frames.append(pd.DataFrame(data, columns=['workbook','worksheet','worksheet_total','worksheet_average']))
    worksheets_data_frame = pd.concat(worksheet_data_frames, axis=0, ignore_index=True)
    
    workbook_total = pd.DataFrame(workbook_total_sales).sum()
    workbook_total_number_of_sales = pd.DataFrame(workbook_number_of_sales).sum()
    workbook_average = pd.DataFrame(workbook_total / workbook_total_number_of_sales)
    
    workbook_stats = {'workbook': os.path.basename(workbook),
                      'workbook_total': workbook_total,
                      'workbook_average': workbook_average}
    
    workbook_stats = pd.DataFrame(workbook_stats, columns=['workbook','workbook_total','workbook_average'])
    workbook_data_frame = pd.merge(worksheets_data_frame, workbook_stats, on='workbook', how='left')
    #계산된 데이터가 있는 worksheets_data_frame과, 분류가 있는 workbook_stats를 workbook 이름 기준으로 left join한다
    
    data_frames.append(workbook_data_frame)
    #아직 파일 별로 각각의 데이터프레임이 존재함

    all_data_concatenated = pd.concat(data_frames, axis=0, ignore_index = True) 
    #세로로 합쳐 준다
    
writer = pd.ExcelWriter(output_file)
all_data_concatenated.to_excel(writer, sheet_name='sums_and_averages', index=False)
writer.save()

pd.read_excel("13pandas_output.xls")

,workbook,worksheet,worksheet_total,worksheet_average,workbook_total,workbook_average
0,sales_2013.xlsx,january_2013,8992,1498.666667,28506,1583.666667
1,sales_2013.xlsx,february_2013,9375,1562.500000,28506,1583.666667
2,sales_2013.xlsx,march_2013,10139,1689.833333,28506,1583.666667
3,sales_2014.xlsx,january_2014,260221,43370.166667,465386,25854.777778
4,sales_2014.xlsx,february_2014,103656,17276.000000,465386,25854.777778
5,sales_2014.xlsx,march_2014,101509,16918.166667,465386,25854.777778
6,sales_2015.xlsx,january_2015,3201,533.500000,304253,16902.944444
7,sales_2015.xlsx,february_2015,55007,9167.833333,304253,16902.944444
8,sales_2015.xlsx,march_2015,246045,41007.500000,304253,16902.944444


## 3.6 연습문제

##### 1. 조건, 집합, 정규 표현식을 기반으로 행을 필터링하는 스크립트 중 하나를 수정하여 예제에서 필터링한 행 집합과 다른 조건의 행 집합을 출력하고 출력 파일에 써보자.

In [409]:
import pandas as pd

input_file = "sales_2013.xlsx"
output_file = "3practice_1.xls"

data_frame = pd.read_excel(input_file, sheetname=None, index_col=None)
#print(data_frame)

filter_date = '2013-02-06'

row_list = []
for worksheet_name, data in data_frame.items():
    #print(data)
    #row_list.append(data[data['Customer ID'] > 1234 ])
    row_list.append(data[data['Purchase Date'] > filter_date])
print(row_list)

all_rows = pd.concat(row_list, axis=0, ignore_index=True)
print('\n',all_rows)

writer = pd.ExcelWriter(output_file)
all_rows.to_excel(writer, sheet_name='DateFilter', index=False)
writer.save()

[Empty DataFrame
Columns: [Customer ID, Customer Name, Invoice Number, Sale Amount, Purchase Date]
Index: [],    Customer ID   Customer Name Invoice Number  Sale Amount Purchase Date
1         8765     Laney Stone       100-0009         1367    2013-02-08
2         7654    Roger Lipney       100-0010         2135    2013-02-15
3         6543   Thomas Haines       100-0011         1346    2013-02-17
4         5432     Anushka Vaz       100-0012         1560    2013-02-21
5         4321  Harriet Cooper       100-0013         1852    2013-02-25,    Customer ID  Customer Name Invoice Number  Sale Amount Purchase Date
0         1234     John Smith       100-0014         1350    2013-03-04
1         8765      Tony Song       100-0015         1167    2013-03-08
2         2345  Mary Harrison       100-0016         1789    2013-03-17
3         6543     Rachel Paz       100-0017         2042    2013-03-22
4         3456     Lucy Gomez       100-0018         1511    2013-03-28
5         4321  Sus

##### 2. 인덱스 값 또는 열 헤더를 기준으로 특정 열을 필터링하는 스크립트 중 하나를 수정하여 예제에서 필터링한 것과 다른 열 집합을 출력하고 출력 파일에 써보자.

In [ ]:
import pandas as pd

input_file = "sales_2015.xlsx"
output_file = "3practice_2.xls"

